In [22]:
import pandas as pd
import numpy as np

In [23]:
data = pd.read_excel('/Users/alessandro/Downloads/ALPHS_FC.xlsx', \
                     sheet_name='FC 3 mth horizon', \
                     header=1)

In [24]:
# count each material group
for mg in set(data['Material Group']):
    count = np.sum(data['Material Group'].values == mg)
    print(mg, count)

ALUPS 27
ALP1L 602
ALPAA 172
ALPSS 722
ALPDU 81
ALPAB 354
ALPHS 780
ALQDU 7
ALPAD 726


In [28]:
filtered_data = data[ (data['Material Group'] == 'ALPHS') & (data['Company code'] == 'BGE')] 

In [29]:
filtered_data

,Company code,Material Group,Horizon (Months),Cal. year / month,ZPC,ZPC.1
80,BGE,ALPHS,3,10.2018,1113.547,1314.086
81,BGE,ALPHS,3,11.2018,1070.887,1070.887
82,BGE,ALPHS,3,12.2018,941.338,786.233
83,BGE,ALPHS,3,1.2019,138.305,356.888
84,BGE,ALPHS,3,2.2019,366.663,395.441
85,BGE,ALPHS,3,3.2019,239.995,314.839
86,BGE,ALPHS,3,4.2019,231.975,211.540
87,BGE,ALPHS,3,5.2019,207.522,190.223
88,BGE,ALPHS,3,6.2019,216.408,207.865
89,BGE,ALPHS,3,7.2019,279.335,202.410


### Exponential Smoothing
s_0 = x_0, s_t = \alpha x_t + (1 - \alpha) s_{t-1}

In [33]:
X = np.round(filtered_data['ZPC'].values)

In [35]:
def exponential_smoothing(x, alpha, s=[]):
    if len(s) == 0:
        s.append(x)
    else:
        st = alpha * x + (1-alpha) * s[-1]
        s.append(st)
    

In [65]:
s = []
alpha = 0.18 # a number between 0 and 1
for x in X:
    exponential_smoothing(x, alpha, s)

In [66]:
# Forecasting the full sequence
print('real', '*grundfos fc*', 'our fc')
for a,b,c in zip(filtered_data['ZPC'].values, filtered_data['ZPC.1'].values, s):
    print(int(np.round(a)), int(np.round(b)), int(np.round(c)))

real *grundfos fc* our fc
1114 1314 1114
1071 1071 1106
941 786 1077
138 357 908
367 395 810
240 315 708
232 212 622
208 190 547
216 208 488
279 202 450
348 158 432
475 475 440
643 830 476
1052 973 580
651 747 593
563 346 587
250 250 527
226 213 472
239 204 430
155 155 381
166 166 342
238 155 323
233 117 307
365 285 318
918 792 426
859 800 504
634 608 527


In [64]:
# Forecasting the full sequence (errors)
print('*grundfos fc*', 'our fc')
for a,b,c in zip(filtered_data['ZPC'].values, filtered_data['ZPC.1'].values, s):
    eb = abs(b-a) / a
    ec = abs(c-a) / a
    print(f"{100*eb:.2f} - {100*ec:.2f}")

*grundfos fc* our fc
18.01 - 0.04
0.00 - 3.22
16.48 - 13.94
158.04 - 540.34
7.85 - 113.25
31.19 - 180.64
8.81 - 152.27
8.34 - 145.65
3.95 - 108.41
27.54 - 49.14
54.69 - 15.87
0.00 - 12.16
29.03 - 28.07
7.46 - 44.82
14.85 - 8.62
38.60 - 4.49
0.00 - 108.38
5.61 - 104.70
14.56 - 74.49
0.00 - 135.65
0.00 - 95.45
34.65 - 29.50
49.85 - 25.54
21.74 - 15.76
13.66 - 53.22
6.88 - 40.04
4.14 - 15.02


### Time window forecasting

In [50]:
s = []
alpha = 0.1 # a number between 0 and 1
for x in X[:5]:
    exponential_smoothing(x, alpha, s)

In [52]:
s_new = [sn for sn in s]

In [57]:
alpha = 0.1 # a number between 0 and 1
for x in X[5:8]:
    exponential_smoothing(x, alpha, s_new)

In [58]:
s_new

[1114.0, 1071.0, 941.0, 138.0, 367.0, 354.3, 342.07, 328.663]